In [1]:
from sympy import *
import numpy as np
import tensorly as tl
from tensorly.decomposition import parafac
from tensorly.decomposition import non_negative_parafac_hals

In [2]:
def low_tensor():
    high = 25
    a = np.random.randint(1, high, size=3) 
    b = np.random.randint(1, high, size=2) 
    c = np.random.randint(1, high, size=2) 
    tens = tl.tensor(np.kron(np.kron(a, b), c).reshape(3, 2, 2)) * 1.0
    return tens


def ntree(p):
    a1 = det(Matrix(p[0]))
    a2 = det(Matrix(p[1]))
    b1 = det(Matrix(p[:,0]))
    b2 = det(Matrix(p[:,1]))
    c1 = det(Matrix(p[:,:,0]))
    c2 = det(Matrix(p[:,:,1]))
    return a1 * a2 >= 0 or b1 * b2 >= 0 or c1 * c2 >= 0

def check_rank(tensor, rank, non_neg=True, n=10, tol=0.001, p=False):
    if non_neg:
        for k in range(n):
            weights, factors = non_negative_parafac_hals(tensor, n_iter_max=1000000, rank=rank, init='random')
            full = tl.cp_to_tensor((weights, factors))
            diff = (full - tensor) / tensor
            if p:
                print(weights,factors)
                print(tl.max(abs(diff)))
            if tl.max(abs(diff)) < tol:
                return True
    else:
        for k in range(n):
            weights, factors = parafac(tensor, n_iter_max=1000000, rank=rank)
            full = tl.cp_to_tensor((weights, factors))
            diff = (full - tensor) / tensor
            if p:
                print(tl.max(abs(diff)))
            if tl.max(abs(diff)) < tol:
                return True
    return False
    

In [11]:
def mac(t):
    a = Matrix(t[0]).reshape(4,1)
    b = Matrix(t[1]).reshape(4,1)
    c = Matrix(t[2]).reshape(4,1)
    d = Matrix(t[3]).reshape(4,1)
    #e = Matrix(t[4]).reshape(4,1)
    M = ones(4,4)
    M[:, 0] = a
    M[:, 1] = b
    M[:, 2] = c
    M[:, 3] = d
    #M[:, 4] = e
    #if abs(M.det()) >= 1:
    #    print(M.det())
    return abs(det(M))< 4

def check_det(p):
    A = tl.tensor([p[0], p[1]])
    B = tl.tensor([p[1], p[2]])
    C = tl.tensor([p[0], p[2]])
    r2 = check_r2(A) or check_r2(B) or check_r2(C)
    return check_r3(A) and check_r3(B) and check_r3(C)

def check_r3(m):
    a = det(Matrix(m[0])) * det(Matrix(m[1]))
    b = det(Matrix(m[:,0])) * det(Matrix(m[:,1]))
    c = det(Matrix(m[:,:,0])) * det(Matrix(m[:,:,1]))
    #print(a,b,c)
    return a >= 0 or b >= 0 or c >= 0

def check_r2(m):
    a = det(Matrix(m[0])) * det(Matrix(m[1]))
    b = det(Matrix(m[:,0])) * det(Matrix(m[:,1]))
    c = det(Matrix(m[:,:,0])) * det(Matrix(m[:,:,1]))
    #print(a * b,b * c, a * b)
    return a >= 0 and b >= 0 and c >= 0

def rt():
    return tl.tensor(np.random.randint(0,10000000, size = (3,2,2))) * 0.03
    

In [12]:
res = {True:0, False:0}
wrong = []
for i in range(1000):
    t1 = low_tensor() * 0.01
    t2 = low_tensor() * 0.01
    t3 = low_tensor() * 0.01
    t4 = low_tensor()
    t5 = low_tensor()
    #t = rt() + rt() + rt() + rt() + rt() / 3 + rt() / 5 + rt() / 7 + rt() / 11 + rt()/ 13 + rt()/ 17 + rt() / 19 + rt() / 23 + rt() / 29 
    t = t1 + t2 + t3
    key = check_det(t)
    res[key] = res[key] + 1
print(res)
print(wrong)   

{True: 1000, False: 0}
[]


In [57]:
p = tl.tensor(np.random.randint(0,10000, size = (3,2,2))) * 1.0 + tl.tensor(np.random.randint(0,10000, size = (3,2,2))) * 1.0 + low_tensor() / 3
check_det(p)

False

In [27]:
A = tl.tensor([p[0], p[1]])
p

array([[[6980., 2711.],
        [1776., 4475.]],

       [[7327., 8708.],
        [4475., 3245.]],

       [[4073., 1663.],
        [6595., 9122.]]])

False